### Feature Extraction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import category_encoders as ce

In [2]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})

order_products_prior = pd.read_csv('order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_train = pd.read_csv( 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})
products = pd.read_csv('products.csv')

 ##### Prepare Data

In [3]:
prior_df = order_products_prior.merge(orders, on = 'order_id', how = 'inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


##### Features Creation

In [4]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id','product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


##### Product Level Features

In [5]:
mean_add_to_cart_order = prior_df.groupby(['product_id']).agg({'add_to_cart_order':'mean'})
total_orders = prior_df.groupby(['product_id']).agg({'reordered':'count'})
total_reorders = prior_df.groupby(['product_id']).agg({'reordered':'sum'})
reorder_per = prior_df.groupby(['product_id']).agg({'reordered':'mean'})
unique_users = prior_df.groupby(['product_id']).agg({'user_id':lambda x: x.nunique()})
order_first_time_total_cnt = prior_df.groupby(['product_id']).agg({'user_buy_product_times':lambda x: sum(x==1)})
order_second_time_total_cnt = prior_df.groupby(['product_id']).agg({'user_buy_product_times':lambda x: sum(x==2)})
is_organic = prior_df.groupby(['product_id']).agg({'product_name': lambda x: 1 if 'Organic' in x else 0})

In [6]:
prod_feats = pd.concat([mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic],axis=1)

In [7]:
prod_feats.columns = ['mean_add_to_cart_order','total_orders','total_reorders','reorder_per','unique_users','order_first_time_total_cnt','order_second_time_total_cnt','is_organic']

In [8]:
prod_feats.reset_index(inplace=True)
prod_feats.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic
0,1,5.801836,1852,1136.0,0.613391,716,716,276,0
1,2,9.888889,90,12.0,0.133333,78,78,8,0
2,3,6.415162,277,203.0,0.732852,74,74,36,0
3,4,9.507599,329,147.0,0.446809,182,182,64,0
4,5,6.466667,15,9.0,0.600000,6,6,4,0


In [9]:
prod_feats['second_time_percent'] = order_second_time_total_cnt/order_first_time_total_cnt

##### Aisle features

In [10]:
aisle_mean_add_to_cart_order = prior_df.groupby(['aisle_id']).agg({'add_to_cart_order':'mean'})
aisle_std_add_to_cart_order = prior_df.groupby(['aisle_id']).agg({'add_to_cart_order':'std'})
aisle_total_orders = prior_df.groupby(['aisle_id']).agg({'reordered':'count'})
aisle_total_reorders = prior_df.groupby(['aisle_id']).agg({'reordered':'sum'})
aisle_reorder_per = prior_df.groupby(['aisle_id']).agg({'reordered':'mean'})
aisle_unique_users = prior_df.groupby(['aisle_id']).agg({'user_id': lambda x: x.nunique()})

In [11]:
aisle_feats = pd.concat([aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_per,aisle_unique_users],axis=1)

In [12]:
aisle_feats.columns = ['aisle_mean_add_to_cart_order','aisle_std_add_to_cart_order','aisle_total_orders','aisle_total_reorders','aisle_reorder_per','aisle_unique_users']

In [13]:
aisle_feats.reset_index(inplace=True)
aisle_feats.head()

,aisle_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_per,aisle_unique_users
0,1,8.167640,7.104166,71928,42912.0,0.596597,20711
1,2,9.275497,7.473802,82491,40365.0,0.489326,31222
2,3,9.571935,7.899672,456386,272922.0,0.598007,63592
3,4,10.161450,7.745705,200687,98243.0,0.489533,53892
4,5,10.297600,8.187047,62510,17542.0,0.280627,32312


##### Department features

In [14]:
department_mean_add_to_cart_order = prior_df.groupby(['department_id']).agg({'add_to_cart_order':'mean'})
department_std_add_to_cart_order = prior_df.groupby(['department_id']).agg({'add_to_cart_order':'std'})
department_total_orders = prior_df.groupby(['department_id']).agg({'reordered':'count'})
department_total_reorders = prior_df.groupby(['department_id']).agg({'reordered':'sum'})
department_reorder_per = prior_df.groupby(['department_id']).agg({'reordered':'mean'})
department_unique_users = prior_df.groupby(['department_id']).agg({'user_id':lambda x: x.nunique()})

In [15]:
dpt_feats = pd.concat([department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,
                      department_total_reorders,department_reorder_per,department_unique_users],axis=1)

In [16]:
dpt_feats.columns = ['department_mean_add_to_cart_order','department_std_add_to_cart_order','department_total_orders',
                      'department_total_reorders','department_reorder_per','department_unique_users']

In [17]:
dpt_feats.reset_index(inplace=True)
dpt_feats.head()

,department_id,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users
0,1,8.996414,7.393502,2236432,1211890.0,0.541885,163233
1,2,8.277645,7.526272,36291,14806.0,0.407980,17875
2,3,8.084397,6.904849,1176787,739188.0,0.628141,140612
3,4,8.022875,6.658899,9479291,6160710.0,0.649913,193237
4,5,5.428346,5.778253,153696,87595.0,0.569924,15798


In [18]:
prod_feats = prod_feats.merge(products,on='product_id',how='left')
prod_feats = prod_feats.merge(aisle_feats,on='aisle_id',how='left')
prod_feats = prod_feats.merge(aisles,on='aisle_id',how='left')
prod_feats = prod_feats.merge(dpt_feats,on='department_id',how='left')
prod_feats = prod_feats.merge(departments,on='department_id',how='left')
prod_feats.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_per,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users,department
0,1,5.801836,1852,1136.0,0.613391,716,716,276,0,NaN,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973.0,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,78,8,0,0.385475,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,74,36,0,0.102564,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892.0,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,182,64,0,0.486486,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890.0,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,6,4,0,0.351648,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry


In [19]:
prod_feats.drop(['product_name','aisle_id','department_id'],axis=1,inplace=True)
prod_feats.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_per,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users,department
0,1,5.801836,1852,1136.0,0.613391,716,716,276,0,NaN,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973.0,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,78,8,0,0.385475,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,74,36,0,0.102564,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892.0,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,182,64,0,0.486486,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890.0,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,6,4,0,0.351648,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry


In [20]:
prod_feats.shape

(49677, 24)

In [21]:
prod_feats.dtypes

product_id                            uint64
mean_add_to_cart_order               float64
total_orders                           int64
total_reorders                       float64
reorder_per                          float64
unique_users                           int64
order_first_time_total_cnt             int64
order_second_time_total_cnt            int64
is_organic                             int64
second_time_percent                  float64
aisle_mean_add_to_cart_order         float64
aisle_std_add_to_cart_order          float64
aisle_total_orders                     int64
aisle_total_reorders                 float64
aisle_reorder_per                    float64
aisle_unique_users                     int64
aisle                                 object
department_mean_add_to_cart_order    float64
department_std_add_to_cart_order     float64
department_total_orders                int64
department_total_reorders            float64
department_reorder_per               float64
department

In [22]:
encoder = ce.BinaryEncoder(cols=['aisle','department'],return_df = True)

In [23]:
prod_feats = encoder.fit_transform(prod_feats)
prod_feats.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136.0,0.613391,716,716,276,0,NaN,...,7.692492,2887550,1657973.0,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,0,0.385475,...,7.875241,1875577,650301.0,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,0,0.102564,...,6.711172,2690129,1757892.0,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,0,0.486486,...,7.393502,2236432,1211890.0,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,0,0.351648,...,7.875241,1875577,650301.0,0.346721,172755,0,0,0,1,0


In [24]:
prod_feats.shape

(49677, 35)

In [25]:
prod_feats.columns

Index(['product_id', 'mean_add_to_cart_order', 'total_orders',
       'total_reorders', 'reorder_per', 'unique_users',
       'order_first_time_total_cnt', 'order_second_time_total_cnt',
       'is_organic', 'second_time_percent', 'aisle_mean_add_to_cart_order',
       'aisle_std_add_to_cart_order', 'aisle_total_orders',
       'aisle_total_reorders', 'aisle_reorder_per', 'aisle_unique_users',
       'aisle_0', 'aisle_1', 'aisle_2', 'aisle_3', 'aisle_4', 'aisle_5',
       'aisle_6', 'aisle_7', 'department_mean_add_to_cart_order',
       'department_std_add_to_cart_order', 'department_total_orders',
       'department_total_reorders', 'department_reorder_per',
       'department_unique_users', 'department_0', 'department_1',
       'department_2', 'department_3', 'department_4'],
      dtype='object')

In [26]:
prod_feats.isnull().any().any()

True

In [27]:
prod_feats.isnull().sum()

product_id                            0
mean_add_to_cart_order                0
total_orders                          0
total_reorders                        0
reorder_per                           0
unique_users                          0
order_first_time_total_cnt            0
order_second_time_total_cnt           0
is_organic                            0
second_time_percent                  12
aisle_mean_add_to_cart_order          0
aisle_std_add_to_cart_order           0
aisle_total_orders                    0
aisle_total_reorders                  0
aisle_reorder_per                     0
aisle_unique_users                    0
aisle_0                               0
aisle_1                               0
aisle_2                               0
aisle_3                               0
aisle_4                               0
aisle_5                               0
aisle_6                               0
aisle_7                               0
department_mean_add_to_cart_order     0


In [28]:
prod_feats.dropna(inplace=True)

In [29]:
prod_feats.isnull().any().any()

False

In [30]:
del aisle_feats,dpt_feats,aisles,departments
gc.collect()

105

##### User level features

In [31]:
prior_df.isnull().any()

order_id                  False
product_id                False
add_to_cart_order         False
reordered                 False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_name              False
aisle_id                  False
department_id             False
user_buy_product_times    False
dtype: bool

In [32]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [33]:
avg_dow = prior_df.groupby(['user_id']).agg({'order_dow':'mean'})
std_dow = prior_df.groupby(['user_id']).agg({'order_dow':'std'})
avg_doh = prior_df.groupby(['user_id']).agg({'order_hour_of_day':'mean'})
std_doh = prior_df.groupby(['user_id']).agg({'order_hour_of_day':'std'})
avg_since_order = prior_df.groupby(['user_id']).agg({'days_since_prior_order':'mean'})
std_since_order = prior_df.groupby(['user_id']).agg({'days_since_prior_order':'std'})
total_orders_by_user = prior_df.groupby(['user_id']).agg({'order_number':lambda x: x.nunique()})
total_products_by_user = prior_df.groupby(['user_id']).agg({'product_id':'count'})
total_unique_product_by_user = prior_df.groupby(['user_id']).agg({'product_id':lambda x: x.nunique()})
total_reorders_by_user = prior_df.groupby(['user_id']).agg({'reordered':'sum'})
reorder_propotion_by_user = prior_df.groupby(['user_id']).agg({'reordered':'mean'})

In [34]:
user_feats = pd.concat([avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,
                      total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user],axis=1)

In [35]:
user_feats.columns = ['avg_dow','std_dow','avg_doh','std_doh','avg_since_order','std_since_order','total_orders_by_user',
                      'total_products_by_user','total_unique_product_by_user','total_reorders_by_user','reorder_propotion_by_user']

In [36]:
user_feats.reset_index(inplace=True)
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,59,18,41.0,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,195,102,93.0,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,88,33,55.0,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,18,17,1.0,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,37,23,14.0,0.378378


##### Additional features -  user level

In [37]:
average_order_size = prior_df.groupby(['user_id','order_number']).agg({'reordered':'count'})
reorder_in_order = prior_df.groupby(['user_id','order_number']).agg({'reordered':'mean'})

In [38]:
user_feats2 = pd.concat([average_order_size,reorder_in_order],axis=1)
user_feats2.columns = ['average_order_size','reorder_in_order']
user_feats2.reset_index(inplace=True)
user_feats2.head()

,user_id,order_number,average_order_size,reorder_in_order
0,1,1,5,0.000
1,1,2,6,0.500
2,1,3,5,0.600
3,1,4,5,1.000
4,1,5,8,0.625


In [39]:
user_feats3 = user_feats2.groupby('user_id').agg({'average_order_size':'mean','reorder_in_order':'mean'})
user_feats3 = user_feats3.reset_index()
user_feats3.head()

,user_id,average_order_size,reorder_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [40]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,59,18,41.0,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,195,102,93.0,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,88,33,55.0,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,18,17,1.0,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,37,23,14.0,0.378378,9.250000,0.377778


##### Additional features -  user level

In [41]:
last_three_orders = user_feats2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13


In [42]:
last_three_orders = user_feats2.merge(last_three_orders,on=['user_id','order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1
0,1,8,6,0.666667,7
1,1,9,6,1.000000,8
2,1,10,9,0.666667,9
3,2,12,19,0.578947,21
4,2,13,9,0.000000,22


In [43]:
last_three_orders['rank'] = last_three_orders.groupby('user_id')['order_number'].rank('dense',ascending=True)

In [44]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], values = ['average_order_size','reorder_in_order']).reset_index(drop=False)
last_order_feats.columns = ['user_id','orders_3','orders_2','orders_1','reorder_3','reorder_2','reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6,6,9,0.666667,1.0,0.666667
1,2,19,9,16,0.578947,0.0,0.625000
2,3,6,5,6,0.833333,1.0,1.000000
3,4,7,2,3,0.142857,0.0,0.000000
4,5,9,5,12,0.444444,0.4,0.666667


In [45]:
user_feats = user_feats.merge(last_order_feats, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


##### User and Product level features

In [46]:
total_product_orders_by_users = prior_df.groupby(['user_id','product_id']).agg({'reordered':'count'})
total_product_reorders_by_users = prior_df.groupby(['user_id','product_id']).agg({'reordered':'sum'})
user_product_reorder_percentage = prior_df.groupby(['user_id','product_id']).agg({'reordered':'mean'})
avg_add_to_cart_by_user = prior_df.groupby(['user_id','product_id']).agg({'add_to_cart_order':'mean'})
avg_days_since_last_bought = prior_df.groupby(['user_id','product_id']).agg({'days_since_prior_order':'mean'})
last_ordered_in = prior_df.groupby(['user_id','product_id']).agg({'order_number':'max'})

In [47]:
user_product_feats = pd.concat([total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,
                               avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in],axis=1)
user_product_feats.columns = ['total_product_orders_by_users','total_product_reorders_by_users','user_product_reorder_percentage',
                               'avg_add_to_cart_by_user','avg_days_since_last_bought','last_ordered_in']
user_product_feats.reset_index(inplace=True)
user_product_feats.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in
0,1,196,10,9,0.900000,1.400000,19.555555,10
1,1,10258,9,8,0.888889,3.333333,19.555555,10
2,1,10326,1,0,0.000000,5.000000,28.000000,5
3,1,12427,10,9,0.900000,3.300000,19.555555,10
4,1,13032,3,2,0.666667,6.333333,21.666666,10


##### Additional features - User and Product Level

In [48]:
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0


In [49]:
last_order = prior_df.merge(last_three_orders, on = ['user_id','order_number'], how = 'inner')
last_order.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,average_order_size,reorder_in_order,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,2,0.000000,2231251,2.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,2,0.000000,2231251,2.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,11,0.818182,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,11,0.818182,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,11,0.818182,282882,1.0


In [50]:
last_order['rank'] = last_order.groupby(['user_id','product_id'])['order_number'].rank('dense',ascending = True)

In [51]:
product_purchase_history = last_order.pivot_table(index = ['user_id','product_id'], columns = 'rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id','product_id','is_reorder_3','is_reorder_2','is_reorder_1']
product_purchase_history.fillna(0,inplace = True)
product_purchase_history.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,0.0,0.0
4,1,25133,1.0,1.0,1.0


In [52]:
user_product_feats = user_product_feats.merge(product_purchase_history, on = ['user_id','product_id'], how = 'left')
user_product_feats.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,19.555555,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,NaN,NaN,NaN
3,1,12427,10,9,0.900000,3.300000,19.555555,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666666,10,1.0,0.0,0.0


In [53]:
user_product_feats.isnull().sum()

user_id                                  0
product_id                               0
total_product_orders_by_users            0
total_product_reorders_by_users          0
user_product_reorder_percentage          0
avg_add_to_cart_by_user                  0
avg_days_since_last_bought          868069
last_ordered_in                          0
is_reorder_3                       8382738
is_reorder_2                       8382738
is_reorder_1                       8382738
dtype: int64

In [54]:
user_product_feats.fillna(0,inplace=True)

### Saving all features

In [55]:
prod_feats.to_pickle('product_features.pkl')
user_feats.to_pickle('user_features.pkl')
user_product_feats.to_pickle('user_product_features.pkl')

In [56]:
df = pd.read_pickle('product_features.pkl')
df.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users,department_0,department_1,department_2,department_3,department_4
1,2,9.888889,90,12.0,0.133333,78,78,8,0,0.385475,...,7.875241,1875577,650301.0,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,0,0.102564,...,6.711172,2690129,1757892.0,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,0,0.486486,...,7.393502,2236432,1211890.0,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,0,0.351648,...,7.875241,1875577,650301.0,0.346721,172755,0,0,0,1,0
5,6,14.125000,8,3.0,0.375000,5,5,2,0,0.666667,...,7.825422,447123,143584.0,0.321129,93969,0,0,1,0,1


In [57]:
df = pd.read_pickle('user_features.pkl')

In [58]:
df.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [59]:
df = pd.read_pickle('user_product_features.pkl')
df.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,19.555555,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,19.555555,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666666,10,1.0,0.0,0.0
